# 1 - Sources des données

## Données tabulaires - SAE

## Web Scrapping - Doctolib

Pour avoir un **indice de la disponibilité des IVGs en France**, nous allons utiliser des données issues du site de prise de rendez-vous médicales le plus utilisé en France, **Doctolib**. Par département et spécialisation IVG, les données extraites sont le nombre de localisations différentes proposant un RDV IVG dans un délai de 14, 30 et 90 jours. Ces données sont accessibles à tous gratuitement en utilisant leur moteur de recherche et des filtres (par [ex](https://www.doctolib.fr/ivg-medicamenteuse/paris?availabilitiesBefore=90&regulationSector%5B%5D=CONTRACTED_1&regulationSector%5B%5D=CONTRACTED_1_WITH_EXTRA&regulationSector%5B%5D=CONTRACTED_1_WITH_OPTAM&regulationSector%5B%5D=CONTRACTED_1_WITH_OPTAM_CO&regulationSector%5B%5D=CONTRACTED_2&regulationSector%5B%5D=CONTRACTED_2_WITH_OPTAM&regulationSector%5B%5D=CONTRACTED_2_WITH_OPTAM_CO&regulationSector%5B%5D=CONTRACTED&regulationSector%5B%5D=CONTRACTED_WITH_EXTRA&regulationSector%5B%5D=ORGANIZATION_CONTRACTED)).

Pour la mise en oeuvre, le script est `scrapper_doctolib.py` qui crée un .csv à l'aboutissement. Le script s'appuie sur la librairie `Playwright` permet de faire du web scrapping sur des sites dynamiques (comme Doctolib qui utilise react et javascript) en utilisant des navigateurs "headless" dans l'arrière plan. Avant de finir sur cette librairie, d'autres options ont été tentées : `BeautifulSoup` (pour adapté au parsing des sites statiques) et `Selenium` (une autre librairie de "headless browsers"). Cependant, `Playwright` a était le plus agile pour la gestion du chargement de la page. De plus, `asyncio` est utilisé pour faire du parallélisme et réduire le temps de scrapping.

> Point législatif
> 
> [Source (CNIL) : ](https://www.cnil.fr/fr/focus-interet-legitime-collecte-par-moissonnage)

Exemple des données obtenues (format dataframe pandas) suite au scraping (environ une demi-heure) :

In [8]:
from scripts.compilation_donnees import *

In [9]:
def importer_tout():
    df_SAE = importer_SAE()
    df_dep = importer_departement()
    df_finess = importer_finess()
    df_drees = importer_drees()
    df_pauv = importer_pauv()

    return df_SAE, df_dep, df_finess, df_drees, df_pauv

def importer_locale(date):
    folder = './donnees/'
    suffix = '_' + date + ".csv"

    df_SAE = pd.read_csv(folder + "SAE" + suffix)
    df_dep = pd.read_csv(folder + "dep" + suffix)
    df_finess = pd.read_csv(folder + "finees" + suffix)
    df_drees = pd.read_csv(folder + "drees" + suffix)
    df_pauv = pd.read_csv(folder + "pauv" + suffix)

    return df_SAE, df_dep, df_finess, df_drees, df_pauv

def sauvegarde_locale():
    df_SAE, df_dep, df_finess, df_drees, df_pauv = importer_tout()
    df_SAE.to_csv(get_path("SAE"), index=False)
    df_dep.to_csv(get_path("dep"), index=False)
    df_finess.to_csv(get_path("finees"), index=False)
    df_drees.to_csv(get_path("drees"), index=False)
    df_pauv.to_csv(get_path("pauv"), index=False)

    return 1

In [10]:
df_SAE, df_dep, df_finess, df_drees, df_pauv = importer_locale("2025-12-12-10-24")

# 2 - Statistiques descriptives 


In [37]:
df_nbcentre = pd.crosstab(df_finess["14"], df_finess["19"])

# df_nbcentre.rename(columns={"14": "Département"}, inplace=True)
df_nbcentre.index.name = "Département"
df_nbcentre = df_nbcentre[["Centre de santÃ© sexuelle", "Centre gratuit d'information de dÃ©pistage et de diagnostic"]]
df_nbcentre.rename(columns={
    'Centre de santÃ© sexuelle': 'Centre de sante sexuelle', 
    "Centre gratuit d'information de dÃ©pistage et de diagnostic" : 'CeGIDD'}, inplace=True)
print(df_nbcentre.head(5))

# à normaliser par nombre d'habitant, pour 1000 habitant (pr avoir un chiffre interprétable)


#Centre de santÃ© sexuelle
#Centre gratuit d'information de dÃ©pistage et de diagnostic

19                       Centre de sante sexuelle  CeGIDD
Département                                              
AIN                                             1       2
AISNE                                           5       3
ALLIER                                          3       3
ALPES DE HAUTE PROVENCE                         0       0
ALPES MARITIMES                                 8       1


In [38]:
# faire la moyenne des délais dans la SAE + nombre de centre qui font des IVG par département 
df_SAE.head(5)
#moy_delais = df_SAE.groupby(dep)[‘Valeur'].mean().reset_index()
# nb centre qui prend en charge les IVG par département 
df_prise_en_charge_dep = (
    df_SAE
    .groupby('dep')['PRIS']
    .sum()
    .reset_index()
)
df_prise_en_charge_dep.head(5)
df_prise_en_charge_dep.rename(columns={'dep': 'DEP'}, inplace=True)
df_dep_ivg = pd.merge(df_prise_en_charge_dep, df_dep, on=['DEP'], how='outer')

df_dep_ivg['Département'] = (
    df_dep_ivg['Département']
        .str.replace('-', ' ', regex=False)
        .str.upper()
)



In [ ]:
df_dep_ivg = pd.merge(df_dep_ivg, df_nbcentre, on=['Département'], how='outer')
df_dep_ivg.head(5)


,DEP,PRIS,REG,Région,Département,PMUN,Centre de sante sexuelle,CeGIDD
0,01,4.0,84.0,Auvergne-Rhône-Alpes,AIN,663202.0,1.0,2.0
1,02,5.0,32.0,Hauts-de-France,AISNE,527468.0,5.0,3.0
2,03,4.0,84.0,Auvergne-Rhône-Alpes,ALLIER,334872.0,3.0,3.0
3,04,2.0,93.0,Provence-Alpes-Côte d'Azur,ALPES DE HAUTE PROVENCE,166077.0,0.0,0.0
4,06,10.0,93.0,Provence-Alpes-Côte d'Azur,ALPES MARITIMES,1103941.0,8.0,1.0


In [ ]:
df_dep_ivg["pop_fe"] = df_dep_ivg["PMUN"] / 2 # population par département divisée par deux pour approximer le nombre de femmes
df_dep_ivg["PRIS_pour1000fe"] = (df_dep_ivg["PRIS"] / df_dep_ivg["pop_fe"])*100000 # nombre de centre qui prennent en charge des IVG pour 1000 femmes
df_dep_ivg["nb_centre_pour1000fe"] = (df_dep_ivg["Centre de sante sexuelle"] / df_dep_ivg["pop_fe"])*100000
df_dep_ivg["nb_cegidd_pour1000fe"] = (df_dep_ivg["CeGIDD"] / df_dep_ivg["pop_fe"])*100000
df_dep_ivg["centres_total"] = df_dep_ivg["Centre de sante sexuelle"] + df_dep_ivg["CeGIDD"]
df_dep_ivg["centres_total_pour1000fe"] = (df_dep_ivg["centres_total"] / df_dep_ivg["pop_fe"])*100000
valeur_min = df_dep_ivg["pop_fe"].min()
print(valeur_min) # 38259 < 100 000 : donc on conserve pr avoir un chiffre interprétable mais pas réaliste pr certains départements


38259.5


,DEP,PRIS,REG,Région,Département,PMUN,Centre de sante sexuelle,CeGIDD,pop_fe,PRIS_pour1000fe,nb_centre_pour1000fe,nb_cegidd_pour1000fe,centres_total,centres_total_pour1000fe
0,01,4.0,84.0,Auvergne-Rhône-Alpes,AIN,663202.0,1.0,2.0,331601.0,1.206269,0.301567,0.603134,3.0,0.904702
1,02,5.0,32.0,Hauts-de-France,AISNE,527468.0,5.0,3.0,263734.0,1.895850,1.895850,1.137510,8.0,3.033359
2,03,4.0,84.0,Auvergne-Rhône-Alpes,ALLIER,334872.0,3.0,3.0,167436.0,2.388973,1.791729,1.791729,6.0,3.583459
3,04,2.0,93.0,Provence-Alpes-Côte d'Azur,ALPES DE HAUTE PROVENCE,166077.0,0.0,0.0,83038.5,2.408521,0.000000,0.000000,0.0,0.000000
4,06,10.0,93.0,Provence-Alpes-Côte d'Azur,ALPES MARITIMES,1103941.0,8.0,1.0,551970.5,1.811691,1.449353,0.181169,9.0,1.630522
